In [3]:
import heapq
import math


class Grid:
    def __init__(self, size):
        self.size = size
        self.start = (0, 0)
        self.goal = (200, 200)

        self.moves = [
            (1, 0, 1), (-1, 0, 1), (0, 1, 1), (0, -1, 1),
            (1, 1, math.sqrt(2)), (1, -1, math.sqrt(2)),
            (-1, 1, math.sqrt(2)), (-1, -1, math.sqrt(2))
        ]

        self.obstacles = self.build_obstacles()

    def build_obstacles(self):
        blocked = set()
        obstacle_regions = [
            ((40, 40), (70, 70)),
            ((90, 120), (130, 160)),
            ((150, 50), (180, 90)),
            ((60, 150), (100, 190))
        ]

        for (x1, y1), (x2, y2) in obstacle_regions:
            for x in range(x1, x2 + 1):
                for y in range(y1, y2 + 1):
                    blocked.add((x, y))
        return blocked

    def in_bounds(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

    def is_valid(self, x, y):
        return self.in_bounds(x, y) and (x, y) not in self.obstacles



class DijkstraSolver:
    def __init__(self, grid):
        self.grid = grid

    def solve(self):
        pq = []
        heapq.heappush(pq, (0, self.grid.start))

        visited = set()
        dist = {self.grid.start: 0}
        expanded = 0

        while pq:
            cost, (x, y) = heapq.heappop(pq)

            if (x, y) in visited:
                continue

            visited.add((x, y))
            expanded += 1

            if (x, y) == self.grid.goal:
                return cost, expanded

            for dx, dy, w in self.grid.moves:
                nx, ny = x + dx, y + dy
                if self.grid.is_valid(nx, ny):
                    new_cost = cost + w
                    if (nx, ny) not in dist or new_cost < dist[(nx, ny)]:
                        dist[(nx, ny)] = new_cost
                        heapq.heappush(pq, (new_cost, (nx, ny)))

        return float('inf'), expanded


class AStarSolver:
    def __init__(self, grid):
        self.grid = grid

    def heuristic(self, x, y):
        return math.sqrt((self.grid.goal[0] - x) ** 2 +
                         (self.grid.goal[1] - y) ** 2)

    def solve(self):
        pq = []
        start_h = self.heuristic(*self.grid.start)
        heapq.heappush(pq, (start_h, 0, self.grid.start))

        visited = set()
        g_cost = {self.grid.start: 0}
        expanded = 0

        while pq:
            f, cost, (x, y) = heapq.heappop(pq)

            if (x, y) in visited:
                continue

            visited.add((x, y))
            expanded += 1

            if (x, y) == self.grid.goal:
                return cost, expanded

            for dx, dy, w in self.grid.moves:
                nx, ny = x + dx, y + dy
                if self.grid.is_valid(nx, ny):
                    new_g = cost + w
                    if (nx, ny) not in g_cost or new_g < g_cost[(nx, ny)]:
                        g_cost[(nx, ny)] = new_g
                        f = new_g + self.heuristic(nx, ny)
                        heapq.heappush(pq, (f, new_g, (nx, ny)))

        return float('inf'), expanded



grid = Grid(201)

dijkstra_solver = DijkstraSolver(grid)
astar_solver = AStarSolver(grid)

d_cost, d_expanded = dijkstra_solver.solve()
a_cost, a_expanded = astar_solver.solve()

print("Dijkstra’s Algorithm:")
print("Path Cost:", d_cost)
print("Expanded Nodes:", d_expanded)

print("\nA* Algorithm (Euclidean Heuristic):")
print("Path Cost:", a_cost)
print("Expanded Nodes:", a_expanded)
print("="*50)
if(a_expanded<d_expanded):
    print("A* is a better algorithm")

Dijkstra’s Algorithm:
Path Cost: 301.002092041054
Expanded Nodes: 34928

A* Algorithm (Euclidean Heuristic):
Path Cost: 301.002092041054
Expanded Nodes: 12239
A* is a better algorithm
